In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [3]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## Rule-Based Bots ##

In [90]:
config = GameConfig(splits=[StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit], sandbox=True, num_players=2, must_include=[])
players = load_players(['BM', 'BM'], None, map_location='cpu', train=False)
agent = players[0]
env = FullEnvironment(config, players, logger=logger)

In [91]:
n = 10000

In [92]:
sim_stats = simulate(env, n, None, turn_log=True, action_log=True)

100%|██████████| 10000/10000 [03:08<00:00, 53.01it/s]
===SUMMARY===
{0: 5716, 1: 5574, 'ProvinceWins': 10000, 'Ties': 1290}


In [95]:
data_path = os.path.join(project_root, 'data', 'bm-bm-10k-stats')

In [96]:
save(data_path, sim_stats)

## Tree Simulations ##

In [6]:
from mcts import GameTree

In [78]:
tree_name = 'r-r-mcts-bm-warm-start-100k-srat-base-default-ucb1-cdecay'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [9]:
tree_name = 'r-r-mcts-bm-warm-start-10k-srat-sandbox-ucb1'
rollout_name = 'default-sandbox-mlog-10k-v2'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [79]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [72]:
tree.selection = 'max'
tree.selection

'max'

In [13]:
tree.skip_level = True

In [131]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
splits = [StartingSplit.Starting34Split, StartingSplit.Starting34Split]
# splits = [StartingSplit.Starting25Split, StartingSplit.Starting25Split]
# config = GameConfig(splits=splits, sandbox=False, num_players=2, must_include=default_kingdom)
config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = load_players(['DW', 'UCT'], [rollout_path], tree=tree, rollout_type='mlog', use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)[0]
# players = [agent, agent]
env = DefaultEnvironment(config, players, logger=logger)

In [81]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [132]:
sim_data = simulate(env, 1000, tree, turn_log=True, action_log=True)

100%|██████████| 1000/1000 [00:43<00:00, 22.97it/s]
===SUMMARY===
{0: 644, 1: 373, 'ProvinceWins': 999, 'Ties': 17}


In [133]:
data_dir = os.path.join(project_root, 'data')
data_name ='dw34-uct34-1k-data'
data_path = os.path.join(data_dir, data_name)

In [134]:
save(data_path, sim_data)